In [1]:
import time
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 200)
from pandas.api.types import is_numeric_dtype
from functools import reduce
from pandas.api.types import is_numeric_dtype
from itertools import groupby

In [2]:
df = pd.read_csv('../data/Iris/Iris-15-attcon.csv')

In [3]:
appr_df = df.copy()

In [4]:
df

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,-,3,1.4,-,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5,-,1.4,0.2,Iris-setosa
5,5.4,3.9,1.7,0.4,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa
7,5,3.4,1.5,0.2,Iris-setosa
8,4.4,2.9,1.4,-,Iris-setosa
9,4.9,3.1,-,0.1,Iris-setosa


### Handling Missing attributes using attribute-concept values !!

#### Defining Goal Set !!

In [5]:
df_headers = list(df)
concept = df_headers[-1]
concept

'class'

In [6]:
#all unique concepts
concept_list = df[concept].unique()
concept_list

array(['Iris-setosa', 'Iris-versicolor', 'Iris-viginica'], dtype=object)

In [7]:
#calculating cases by concepts and making sets
U = [] #universal list containing all cases
temp_list = []
goal_list = []
for item in concept_list:
    for index, row in df.iterrows():
        U.append(index+1)
        if row[concept] == item:
            temp_list.append(index)
    goal_list.append(temp_list)
    temp_list = []

In [8]:
print(goal_list,)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49], [50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99], [100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149]]


#### Building Case List !!

In [9]:
attributes = list(df)
attributes

['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']

In [10]:
case_list = []

In [11]:
def discretize(numeric_col):
    print(numeric_col)
    #Sorting the values of numeric column
    sort_col = df[numeric_col].sort_values()
   
    df['sort_col'] = sort_col.values
    point_list = df['sort_col'].unique()
    point_list = list(point_list)
    if '-' in point_list:
        point_list.remove('-')
    print(point_list)
    
    #Finding average between each two points
    avg_list = []
    for i in range(len(point_list)-1):
        avg = (float(point_list[i]) + float(point_list[i+1]))/2
        avg_list.append(round(float(avg),1))
        
    print (avg_list)
    #Performing the discretization and adding the cases
    for i in avg_list:
        case = str(numeric_col) + "," + str(round(float(point_list[0]),1)) + ".." + str(i)
        case2 = str(numeric_col) + "," + str(i) + ".." + str(round(float(point_list[len(point_list)-1]),1))
        case_list.append(case)
        case_list.append(case2)

In [12]:
#Discretization considering upto 2 decimal point
for item in attributes[:-1]:
        discretize(item)

sepal_length
['4.3', '4.4', '4.5', '4.6', '4.7', '4.8', '4.9', '5', '5.1', '5.2', '5.3', '5.4', '5.5', '5.6', '5.7', '5.8', '5.9', '6', '6.1', '6.2', '6.3', '6.4', '6.5', '6.6', '6.7', '6.8', '6.9', '7', '7.1', '7.2', '7.3', '7.4', '7.6', '7.7', '7.9']
[4.3, 4.5, 4.5, 4.7, 4.8, 4.8, 5.0, 5.0, 5.2, 5.2, 5.3, 5.5, 5.5, 5.7, 5.8, 5.8, 6.0, 6.0, 6.2, 6.2, 6.3, 6.5, 6.5, 6.7, 6.8, 6.8, 7.0, 7.0, 7.2, 7.2, 7.3, 7.5, 7.7, 7.8]
sepal_width
['2.2', '2.3', '2.4', '2.5', '2.6', '2.7', '2.8', '2.9', '3', '3.1', '3.2', '3.3', '3.4', '3.5', '3.6', '3.7', '3.8', '3.9', '4', '4.1', '4.2', '4.4']
[2.2, 2.3, 2.5, 2.5, 2.7, 2.8, 2.8, 3.0, 3.0, 3.2, 3.2, 3.3, 3.5, 3.5, 3.7, 3.8, 3.8, 4.0, 4.0, 4.2, 4.3]
petal_length
['1', '1.1', '1.2', '1.3', '1.4', '1.5', '1.6', '1.7', '1.9', '3', '3.3', '3.5', '3.6', '3.7', '3.8', '3.9', '4', '4.1', '4.2', '4.3', '4.4', '4.5', '4.6', '4.7', '4.8', '4.9', '5', '5.1', '5.2', '5.3', '5.4', '5.5', '5.6', '5.7', '5.8', '5.9', '6', '6.1', '6.3', '6.4', '6.6', '6.7', '6.9']
[1

In [13]:
case_list

['sepal_length,4.3..4.3',
 'sepal_length,4.3..7.9',
 'sepal_length,4.3..4.5',
 'sepal_length,4.5..7.9',
 'sepal_length,4.3..4.5',
 'sepal_length,4.5..7.9',
 'sepal_length,4.3..4.7',
 'sepal_length,4.7..7.9',
 'sepal_length,4.3..4.8',
 'sepal_length,4.8..7.9',
 'sepal_length,4.3..4.8',
 'sepal_length,4.8..7.9',
 'sepal_length,4.3..5.0',
 'sepal_length,5.0..7.9',
 'sepal_length,4.3..5.0',
 'sepal_length,5.0..7.9',
 'sepal_length,4.3..5.2',
 'sepal_length,5.2..7.9',
 'sepal_length,4.3..5.2',
 'sepal_length,5.2..7.9',
 'sepal_length,4.3..5.3',
 'sepal_length,5.3..7.9',
 'sepal_length,4.3..5.5',
 'sepal_length,5.5..7.9',
 'sepal_length,4.3..5.5',
 'sepal_length,5.5..7.9',
 'sepal_length,4.3..5.7',
 'sepal_length,5.7..7.9',
 'sepal_length,4.3..5.8',
 'sepal_length,5.8..7.9',
 'sepal_length,4.3..5.8',
 'sepal_length,5.8..7.9',
 'sepal_length,4.3..6.0',
 'sepal_length,6.0..7.9',
 'sepal_length,4.3..6.0',
 'sepal_length,6.0..7.9',
 'sepal_length,4.3..6.2',
 'sepal_length,6.2..7.9',
 'sepal_leng

#### Fill up missing values !!

In [14]:
df2 = df.copy()

In [15]:
for col in df:
    for i, row_value in df[col].iteritems():
        
        #This is for attribute-concept value interpretation
        #For Lost value interpretation, we don't add that case to any block at all
        if row_value == '-':
            
            final_values = []
            print(col,i)
            concept_tmp = df[concept][i]
            print(concept_tmp)
            
            for item_list in goal_list:
                if i in item_list:
                    final_values = item_list
                    
            print (final_values)
            
            #Taking out all unique cases corresponding to the matched concept
            col_values = []
            for item in final_values:
                col_values.append(df[col][item])
        
            
            uniqueList = []
            for letter in col_values:
                if letter not in uniqueList:
                    uniqueList.append(letter)
            
            if '-' in uniqueList:
                uniqueList.remove('-')
                
            if '?' in uniqueList:
                uniqueList.remove('?')
            print(uniqueList)
            
#             print(df.loc[i,col])
            df2.at[i, col] = uniqueList
            
    print("\n")

sepal_length 1
Iris-setosa
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
['5.1', '4.7', '4.6', '5', '5.4', '4.4', '4.9', '4.8', '4.3', '5.8', '5.7', '5.2', '5.5', '4.5', '5.3']
sepal_length 18
Iris-setosa
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
['5.1', '4.7', '4.6', '5', '5.4', '4.4', '4.9', '4.8', '4.3', '5.8', '5.7', '5.2', '5.5', '4.5', '5.3']
sepal_length 42
Iris-setosa
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
['5.1', '4.7', '4.6', '5', '5.4', '4.4', '4.9', '4.8', '4.3', '5.8', '5.7', '5.2', '5.5', '4.5', '5.3']
sepal_length 51
Iris-versicolor


In [16]:
df2

,sepal_length,sepal_width,petal_length,petal_width,class,sort_col
0,5.1,3.5,1.4,0.2,Iris-setosa,"[0.1, 0.2, 0.3]"
1,"[5.1, 4.7, 4.6, 5, 5.4, 4.4, 4.9, 4.8, 4.3, 5....",3,1.4,"[0.2, 0.4, 0.3, 0.1, 0.5, 0.6]",Iris-setosa,"[0.1, 0.2, 0.3]"
2,4.7,3.2,1.3,0.2,Iris-setosa,"[0.1, 0.2, 0.3]"
3,4.6,3.1,1.5,0.2,Iris-setosa,"[0.1, 0.2, 0.3]"
4,5,"[3.5, 3, 3.2, 3.1, 3.9, 3.4, 2.9, 3.7, 4, 4.4,...",1.4,0.2,Iris-setosa,"[0.1, 0.2, 0.3]"
5,5.4,3.9,1.7,0.4,Iris-setosa,"[0.1, 0.2, 0.3]"
6,4.6,3.4,1.4,0.3,Iris-setosa,"[0.1, 0.2, 0.3]"
7,5,3.4,1.5,0.2,Iris-setosa,"[0.1, 0.2, 0.3]"
8,4.4,2.9,1.4,"[0.2, 0.4, 0.3, 0.1, 0.5, 0.6]",Iris-setosa,"[0.1, 0.2, 0.3]"
9,4.9,3.1,"[1.4, 1.3, 1.5, 1.7, 1.1, 1.2, 1, 1.9, 1.6]",0.1,Iris-setosa,"[0.1, 0.2, 0.3]"


In [17]:
df = df2.copy()

In [18]:
df_headers = list(df)
concept = df_headers[-2]
concept_list = df[concept].unique()

In [19]:
concept_list

array(['Iris-setosa', 'Iris-versicolor', 'Iris-viginica'], dtype=object)

In [20]:
#calculating cases by concepts and making sets
temp_list = []
goal_list = []
for item in concept_list:
    for index, row in df.iterrows():
        if row[concept] == item:
            temp_list.append(index+1)
    goal_list.append(temp_list)
    temp_list = []

In [21]:
print(goal_list,)

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50], [51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100], [101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150]]


In [22]:
df

,sepal_length,sepal_width,petal_length,petal_width,class,sort_col
0,5.1,3.5,1.4,0.2,Iris-setosa,"[0.1, 0.2, 0.3]"
1,"[5.1, 4.7, 4.6, 5, 5.4, 4.4, 4.9, 4.8, 4.3, 5....",3,1.4,"[0.2, 0.4, 0.3, 0.1, 0.5, 0.6]",Iris-setosa,"[0.1, 0.2, 0.3]"
2,4.7,3.2,1.3,0.2,Iris-setosa,"[0.1, 0.2, 0.3]"
3,4.6,3.1,1.5,0.2,Iris-setosa,"[0.1, 0.2, 0.3]"
4,5,"[3.5, 3, 3.2, 3.1, 3.9, 3.4, 2.9, 3.7, 4, 4.4,...",1.4,0.2,Iris-setosa,"[0.1, 0.2, 0.3]"
5,5.4,3.9,1.7,0.4,Iris-setosa,"[0.1, 0.2, 0.3]"
6,4.6,3.4,1.4,0.3,Iris-setosa,"[0.1, 0.2, 0.3]"
7,5,3.4,1.5,0.2,Iris-setosa,"[0.1, 0.2, 0.3]"
8,4.4,2.9,1.4,"[0.2, 0.4, 0.3, 0.1, 0.5, 0.6]",Iris-setosa,"[0.1, 0.2, 0.3]"
9,4.9,3.1,"[1.4, 1.3, 1.5, 1.7, 1.1, 1.2, 1, 1.9, 1.6]",0.1,Iris-setosa,"[0.1, 0.2, 0.3]"


#### Building blocks for attribute-value pair !!

In [23]:
temp_list = []
att_val_list = []
for item in case_list:
    a,b = item.split(",") #a = attribute and b = value
    if ".." in b:
        start,end = b.split("..")
        for index, row in df.iterrows():
            if type(row[a]) == list:
                tmp_list = row[a]
                for case in tmp_list:
                    if float(case) >= float(start) and float(case) <= float(end):
                        temp_list.append(index+1)
                   
                
            else:
                if float(row[a]) >= float(start) and float(row[a]) <= float(end):
                    temp_list.append(index+1)
                
        temp_list = list(set(temp_list))
        att_val_list.append(temp_list)
        temp_list = []
        
    else:
        for index, row in df.iterrows():
            if type(row[a]) == list:
                tmp_list = row[a]
                for case in tmp_list:
                    if float(case) == b:
                        temp_list.append(index+1)
            else:           
                if float(row[a]) == b:
                    temp_list.append(index+1)
        
        temp_list = list(set(temp_list))
        att_val_list.append(temp_list)
        temp_list = []

In [24]:
#Print two evenly distributed list side by side
fmt = '%-8s%-20s%s'

for i, (case, att_val) in enumerate(zip(case_list, att_val_list)):
    print(fmt % (i, case, att_val))

0       sepal_length,4.3..4.3[43, 2, 19, 14]
1       sepal_length,4.3..7.9[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150]
2       sepal_length,4.3..4.5[2, 39, 9, 42, 43, 14, 19]
3       sepal_length,4.5..7.9[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 5

In [25]:
#Creating data for case and att-value list
data = {'Cases': case_list, 'att_val': att_val_list}

In [26]:
df2 = pd.DataFrame(data)

In [27]:
#Cases and corresponding att-value pairs
df2

,Cases,att_val
0,"sepal_length,4.3..4.3","[43, 2, 19, 14]"
1,"sepal_length,4.3..7.9","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,"sepal_length,4.3..4.5","[2, 39, 9, 42, 43, 14, 19]"
3,"sepal_length,4.5..7.9","[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 15, 1..."
4,"sepal_length,4.3..4.5","[2, 39, 9, 42, 43, 14, 19]"
5,"sepal_length,4.5..7.9","[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 15, 1..."
6,"sepal_length,4.3..4.7","[2, 3, 4, 7, 39, 9, 42, 43, 14, 48, 19, 23, 30]"
7,"sepal_length,4.7..7.9","[1, 2, 3, 5, 6, 8, 10, 11, 12, 13, 15, 16, 17,..."
8,"sepal_length,4.3..4.8","[2, 3, 4, 7, 39, 9, 42, 43, 12, 13, 14, 46, 48..."
9,"sepal_length,4.8..7.9","[1, 2, 5, 6, 8, 10, 11, 12, 13, 15, 16, 17, 18..."


### Start of Lower/Upper & Probabilistic Approximations !!

#### Building Chacteristic Set !!

In [28]:
attributes = list(appr_df)
del attributes[-1]

In [29]:
attributes

['sepal_length', 'sepal_width', 'petal_length', 'petal_width']

In [30]:
appr_df

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,-,3,1.4,-,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5,-,1.4,0.2,Iris-setosa
5,5.4,3.9,1.7,0.4,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa
7,5,3.4,1.5,0.2,Iris-setosa
8,4.4,2.9,1.4,-,Iris-setosa
9,4.9,3.1,-,0.1,Iris-setosa


In [31]:
case_list = []
#Loop through all the attributes except last 2 - Concept and sort_col
for item in attributes:
    print(item)
    
    #check for non numeric columns
    if not is_numeric_dtype(appr_df[item]):
        temp = appr_df[item].unique()
        print(temp)
        for i in temp:
            if i == '?' or i == '-':
                continue
            else:
                case = item + "," + i
                case_list.append(case)

sepal_length
['5.1' '-' '4.7' '4.6' '5' '5.4' '4.4' '4.9' '4.8' '4.3' '5.8' '5.7' '5.2'
 '5.5' '4.5' '5.3' '7' '6.9' '6.5' '6.3' '6.6' '5.9' '6' '6.1' '5.6' '6.7'
 '6.2' '6.4' '6.8' '7.1' '7.6' '7.3' '7.2' '7.7' '7.4' '7.9']
sepal_width
['3.5' '3' '3.2' '3.1' '-' '3.9' '3.4' '2.9' '3.7' '4' '4.4' '3.8' '3.6'
 '3.3' '4.1' '4.2' '2.3' '2.8' '2.4' '2.7' '2.2' '2.5' '2.6']
petal_length
['1.4' '1.3' '1.5' '1.7' '-' '1.1' '1.2' '1' '1.9' '1.6' '4.7' '4.5' '4.9'
 '4' '4.6' '3.3' '3.9' '3.5' '4.2' '3.6' '4.4' '4.1' '4.8' '4.3' '5' '3.8'
 '3.7' '5.1' '3' '6' '5.9' '5.6' '6.6' '6.3' '5.8' '5.3' '5.5' '6.7' '6.9'
 '5.7' '6.1' '6.4' '5.4' '5.2']
petal_width
['0.2' '-' '0.4' '0.3' '0.1' '0.5' '0.6' '1.5' '1.3' '1.6' '1' '1.4' '1.1'
 '1.8' '1.2' '2.5' '1.9' '2.1' '2.2' '1.7' '2' '2.4' '2.3']


In [32]:
case_list

['sepal_length,5.1',
 'sepal_length,4.7',
 'sepal_length,4.6',
 'sepal_length,5',
 'sepal_length,5.4',
 'sepal_length,4.4',
 'sepal_length,4.9',
 'sepal_length,4.8',
 'sepal_length,4.3',
 'sepal_length,5.8',
 'sepal_length,5.7',
 'sepal_length,5.2',
 'sepal_length,5.5',
 'sepal_length,4.5',
 'sepal_length,5.3',
 'sepal_length,7',
 'sepal_length,6.9',
 'sepal_length,6.5',
 'sepal_length,6.3',
 'sepal_length,6.6',
 'sepal_length,5.9',
 'sepal_length,6',
 'sepal_length,6.1',
 'sepal_length,5.6',
 'sepal_length,6.7',
 'sepal_length,6.2',
 'sepal_length,6.4',
 'sepal_length,6.8',
 'sepal_length,7.1',
 'sepal_length,7.6',
 'sepal_length,7.3',
 'sepal_length,7.2',
 'sepal_length,7.7',
 'sepal_length,7.4',
 'sepal_length,7.9',
 'sepal_width,3.5',
 'sepal_width,3',
 'sepal_width,3.2',
 'sepal_width,3.1',
 'sepal_width,3.9',
 'sepal_width,3.4',
 'sepal_width,2.9',
 'sepal_width,3.7',
 'sepal_width,4',
 'sepal_width,4.4',
 'sepal_width,3.8',
 'sepal_width,3.6',
 'sepal_width,3.3',
 'sepal_width,4

In [33]:
temp_list = []
att_val_list = []
for item in case_list:
    a,b = item.split(",") #a = attribute and b = value
    if ".." in b:
        start,end = b.split("..")
        for index, row in appr_df.iterrows():
            if row[a] >= float(start) and row[a] <= float(end):
                temp_list.append(index+1)
        print(temp_list)
        att_val_list.append(temp_list)
        temp_list = []
        
    else:
        for index, row in appr_df.iterrows():
            if type(row[a]) == list:
                tmp_list = row[a]
                for case in tmp_list:
                    if case == b:
                        temp_list.append(index+1)
                        
            if row[a] == b:
                temp_list.append(index+1)
        print(temp_list)
        att_val_list.append(temp_list)
        temp_list = []

[1, 18, 20, 22, 24, 40, 45, 47, 99]
[3, 30]
[4, 7, 23, 48]
[5, 8, 26, 27, 36, 41, 44, 50, 61, 94]
[6, 11, 17, 21, 32, 85]
[9, 39]
[10, 35, 38, 58, 107]
[12, 13, 25, 31, 46]
[14]
[15, 68, 83, 93, 102, 115, 143]
[16, 56, 80, 97, 100]
[28, 29, 33, 60]
[34, 37, 54, 81, 82, 90, 91]
[42]
[49]
[51]
[53, 121, 140]
[55, 105, 111, 117, 148]
[57, 73, 88, 101, 104, 124, 134, 137, 147]
[59, 76]
[62, 71, 150]
[63, 79, 84, 86, 120, 139]
[64, 72, 92, 128, 135]
[65, 67, 89, 95, 122]
[66, 78, 87, 109, 125, 141, 145, 146]
[69, 98, 127, 149]
[75, 112, 129, 138]
[77, 113, 144]
[103]
[106]
[108]
[110, 126, 130]
[118, 119, 123, 136]
[131]
[132]
[1, 18, 37, 41, 44]
[2, 13, 14, 26, 39, 46, 62, 78, 89, 92, 105, 106, 113, 128, 130, 136, 139, 146, 148, 150]
[3, 30, 36, 43, 48, 51, 52, 71, 111, 121, 126, 144]
[4, 10, 31, 35, 38, 53, 66, 87, 138, 140, 141, 142]
[6, 17]
[7, 8, 12, 21, 25, 27, 29, 32, 86, 137, 149]
[9, 59, 64, 65, 75, 79, 97, 98, 104, 108]
[11, 22, 49]
[15]
[16]
[19, 20, 118, 132]
[23, 110]
[24, 50, 

In [34]:
df = df.drop(columns=['sort_col'])

In [35]:
df

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,"[5.1, 4.7, 4.6, 5, 5.4, 4.4, 4.9, 4.8, 4.3, 5....",3,1.4,"[0.2, 0.4, 0.3, 0.1, 0.5, 0.6]",Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5,"[3.5, 3, 3.2, 3.1, 3.9, 3.4, 2.9, 3.7, 4, 4.4,...",1.4,0.2,Iris-setosa
5,5.4,3.9,1.7,0.4,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa
7,5,3.4,1.5,0.2,Iris-setosa
8,4.4,2.9,1.4,"[0.2, 0.4, 0.3, 0.1, 0.5, 0.6]",Iris-setosa
9,4.9,3.1,"[1.4, 1.3, 1.5, 1.7, 1.1, 1.2, 1, 1.9, 1.6]",0.1,Iris-setosa


In [36]:
U = set(U)
print(U,)

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150}


In [37]:
# Creating dictionary combining case_list and att_val list
block = dict(zip(case_list, att_val_list))
block

{'sepal_length,5.1': [1, 18, 20, 22, 24, 40, 45, 47, 99],
 'sepal_length,4.7': [3, 30],
 'sepal_length,4.6': [4, 7, 23, 48],
 'sepal_length,5': [5, 8, 26, 27, 36, 41, 44, 50, 61, 94],
 'sepal_length,5.4': [6, 11, 17, 21, 32, 85],
 'sepal_length,4.4': [9, 39],
 'sepal_length,4.9': [10, 35, 38, 58, 107],
 'sepal_length,4.8': [12, 13, 25, 31, 46],
 'sepal_length,4.3': [14],
 'sepal_length,5.8': [15, 68, 83, 93, 102, 115, 143],
 'sepal_length,5.7': [16, 56, 80, 97, 100],
 'sepal_length,5.2': [28, 29, 33, 60],
 'sepal_length,5.5': [34, 37, 54, 81, 82, 90, 91],
 'sepal_length,4.5': [42],
 'sepal_length,5.3': [49],
 'sepal_length,7': [51],
 'sepal_length,6.9': [53, 121, 140],
 'sepal_length,6.5': [55, 105, 111, 117, 148],
 'sepal_length,6.3': [57, 73, 88, 101, 104, 124, 134, 137, 147],
 'sepal_length,6.6': [59, 76],
 'sepal_length,5.9': [62, 71, 150],
 'sepal_length,6': [63, 79, 84, 86, 120, 139],
 'sepal_length,6.1': [64, 72, 92, 128, 135],
 'sepal_length,5.6': [65, 67, 89, 95, 122],
 'sepal

In [38]:
attributes = list(df)
attributes

['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']

In [39]:
dic = {}
for index, row in df.iterrows():
    tmp_set = set()
    final_union = []
    char_list = []
    char_list_2 = []
    final_union_set = []
  
    for cols in attributes[:-1]:
        #If the value for corresponding attribute is a list then create all of the att-value pairs
        if type(df.loc[index,cols]) == list:
            print("When values are list") #########
            for item in df.loc[index,cols]:
                block_key = cols + "," + item
                char_list.append(block_key) #char_list has all att-val cases
                print(char_list) #########
                
            union_set = set()
            #Compute union of att-concept value case
            for item in char_list:
                union_set = union_set.union(set(block[item]))
            
            print("Union Set: ", union_set) #########
            final_union.append(union_set)
        
        else:
            print("When value is single") #########
            block_key = cols + "," + df.loc[index,cols]
            char_list_2.append(block_key) #char_list_2 has all single cases
            print(char_list_2) ##########
   
    #Compute instersection for this current row for Characteristics set
    
    print("final_union: ", final_union) ########
    if len(final_union):
        final_union_set = list(reduce(set.intersection, [set(item) for item in final_union]))
        
    
    print("Final Union Set: ", final_union_set) ##########
    
    for item in char_list_2:
        if item in block:
            print("When item is found as key in the block") ########
            if tmp_set == set():
                #Copy over the current set elements to B
                for i in range(len(block[item])):
                    tmp_set.add(block[item][i])
                    
            tmp_set = tmp_set.intersection(set(block[item]))
            print(tmp_set) ########
        
        #If item not in block
        else:
            print("When item is not found as key in the block") ########
            print(tmp_set) ########
            print(U) #########
            if tmp_set == set():
                tmp_set = U
                
            tmp_set = tmp_set.intersection(U)
            print(tmp_set) ########
    
    print("Final tmp_set: ", tmp_set) #########
    final_union_set = set(final_union_set)
    
    if final_union_set == set():
        tmp_set = tmp_set
    else:
        tmp_set = tmp_set.intersection(final_union_set)
        
    print("This is the final value: ", tmp_set) ###########
        
    key = ('K_%d' % (index+1))
    print(key) #########
    dic[key] = tmp_set
    print("\n") ##########

When value is single
['sepal_length,5.1']
When value is single
['sepal_length,5.1', 'sepal_width,3.5']
When value is single
['sepal_length,5.1', 'sepal_width,3.5', 'petal_length,1.4']
When value is single
['sepal_length,5.1', 'sepal_width,3.5', 'petal_length,1.4', 'petal_width,0.2']
final_union:  []
Final Union Set:  []
When item is found as key in the block
{1, 99, 40, 45, 47, 18, 20, 22, 24}
When item is found as key in the block
{1, 18}
When item is found as key in the block
{1, 18}
When item is found as key in the block
{1}
Final tmp_set:  {1}
This is the final value:  {1}
K_1


When values are list
['sepal_length,5.1']
['sepal_length,5.1', 'sepal_length,4.7']
['sepal_length,5.1', 'sepal_length,4.7', 'sepal_length,4.6']
['sepal_length,5.1', 'sepal_length,4.7', 'sepal_length,4.6', 'sepal_length,5']
['sepal_length,5.1', 'sepal_length,4.7', 'sepal_length,4.6', 'sepal_length,5', 'sepal_length,5.4']
['sepal_length,5.1', 'sepal_length,4.7', 'sepal_length,4.6', 'sepal_length,5', 'sepal_le

Final tmp_set:  {21}
This is the final value:  {21}
K_21


When value is single
['sepal_length,5.1']
When value is single
['sepal_length,5.1', 'sepal_width,3.7']
When value is single
['sepal_length,5.1', 'sepal_width,3.7', 'petal_length,1.5']
When value is single
['sepal_length,5.1', 'sepal_width,3.7', 'petal_length,1.5', 'petal_width,0.4']
final_union:  []
Final Union Set:  []
When item is found as key in the block
{1, 99, 40, 45, 47, 18, 20, 22, 24}
When item is found as key in the block
{22}
When item is found as key in the block
{22}
When item is found as key in the block
{22}
Final tmp_set:  {22}
This is the final value:  {22}
K_22


When value is single
['sepal_length,4.6']
When value is single
['sepal_length,4.6', 'sepal_width,3.6']
When value is single
['sepal_length,4.6', 'sepal_width,3.6', 'petal_length,1']
When value is single
['sepal_length,4.6', 'sepal_width,3.6', 'petal_length,1', 'petal_width,0.2']
final_union:  []
Final Union Set:  []
When item is found as key in the bl

['sepal_length,5', 'sepal_width,3.3', 'petal_length,1.4']
When value is single
['sepal_length,5', 'sepal_width,3.3', 'petal_length,1.4', 'petal_width,0.2']
final_union:  []
Final Union Set:  []
When item is found as key in the block
{36, 5, 8, 41, 44, 50, 26, 27, 61, 94}
When item is found as key in the block
{50}
When item is found as key in the block
{50}
When item is found as key in the block
{50}
Final tmp_set:  {50}
This is the final value:  {50}
K_50


When value is single
['sepal_length,7']
When value is single
['sepal_length,7', 'sepal_width,3.2']
When value is single
['sepal_length,7', 'sepal_width,3.2', 'petal_length,4.7']
When values are list
['petal_width,1.5']
['petal_width,1.5', 'petal_width,1.3']
['petal_width,1.5', 'petal_width,1.3', 'petal_width,1.6']
['petal_width,1.5', 'petal_width,1.3', 'petal_width,1.6', 'petal_width,1']
['petal_width,1.5', 'petal_width,1.3', 'petal_width,1.6', 'petal_width,1', 'petal_width,1.4']
['petal_width,1.5', 'petal_width,1.3', 'petal_width,

final_union:  []
Final Union Set:  []
When item is found as key in the block
{66, 109, 78, 141, 145, 146, 87, 125}
When item is found as key in the block
{66, 141, 87}
When item is found as key in the block
{66}
When item is found as key in the block
{66}
Final tmp_set:  {66}
This is the final value:  {66}
K_66


When value is single
['sepal_length,5.6']
When values are list
['sepal_width,3.2']
['sepal_width,3.2', 'sepal_width,3.1']
['sepal_width,3.2', 'sepal_width,3.1', 'sepal_width,2.8']
['sepal_width,3.2', 'sepal_width,3.1', 'sepal_width,2.8', 'sepal_width,2.4']
['sepal_width,3.2', 'sepal_width,3.1', 'sepal_width,2.8', 'sepal_width,2.4', 'sepal_width,2.9']
['sepal_width,3.2', 'sepal_width,3.1', 'sepal_width,2.8', 'sepal_width,2.4', 'sepal_width,2.9', 'sepal_width,2.7']
['sepal_width,3.2', 'sepal_width,3.1', 'sepal_width,2.8', 'sepal_width,2.4', 'sepal_width,2.9', 'sepal_width,2.7', 'sepal_width,3']
['sepal_width,3.2', 'sepal_width,3.1', 'sepal_width,2.8', 'sepal_width,2.4', 'sepal_w

Final Union Set:  []
When item is found as key in the block
{97, 100, 16, 80, 56}
When item is found as key in the block
{97}
When item is found as key in the block
{97}
When item is found as key in the block
{97}
Final tmp_set:  {97}
This is the final value:  {97}
K_97


When value is single
['sepal_length,6.2']
When value is single
['sepal_length,6.2', 'sepal_width,2.9']
When value is single
['sepal_length,6.2', 'sepal_width,2.9', 'petal_length,4.3']
When value is single
['sepal_length,6.2', 'sepal_width,2.9', 'petal_length,4.3', 'petal_width,1.3']
final_union:  []
Final Union Set:  []
When item is found as key in the block
{98, 69, 149, 127}
When item is found as key in the block
{98}
When item is found as key in the block
{98}
When item is found as key in the block
{98}
Final tmp_set:  {98}
This is the final value:  {98}
K_98


When value is single
['sepal_length,5.1']
When value is single
['sepal_length,5.1', 'sepal_width,2.5']
When value is single
['sepal_length,5.1', 'sepal_widt

When values are list
['sepal_length,6.3']
['sepal_length,6.3', 'sepal_length,5.8']
['sepal_length,6.3', 'sepal_length,5.8', 'sepal_length,7.1']
['sepal_length,6.3', 'sepal_length,5.8', 'sepal_length,7.1', 'sepal_length,6.5']
['sepal_length,6.3', 'sepal_length,5.8', 'sepal_length,7.1', 'sepal_length,6.5', 'sepal_length,7.6']
['sepal_length,6.3', 'sepal_length,5.8', 'sepal_length,7.1', 'sepal_length,6.5', 'sepal_length,7.6', 'sepal_length,4.9']
['sepal_length,6.3', 'sepal_length,5.8', 'sepal_length,7.1', 'sepal_length,6.5', 'sepal_length,7.6', 'sepal_length,4.9', 'sepal_length,7.3']
['sepal_length,6.3', 'sepal_length,5.8', 'sepal_length,7.1', 'sepal_length,6.5', 'sepal_length,7.6', 'sepal_length,4.9', 'sepal_length,7.3', 'sepal_length,6.7']
['sepal_length,6.3', 'sepal_length,5.8', 'sepal_length,7.1', 'sepal_length,6.5', 'sepal_length,7.6', 'sepal_length,4.9', 'sepal_length,7.3', 'sepal_length,6.7', 'sepal_length,7.2']
['sepal_length,6.3', 'sepal_length,5.8', 'sepal_length,7.1', 'sepal_le

{102, 143}
Final tmp_set:  {102, 143}
This is the final value:  {102, 143}
K_143


When value is single
['sepal_length,6.8']
When value is single
['sepal_length,6.8', 'sepal_width,3.2']
When value is single
['sepal_length,6.8', 'sepal_width,3.2', 'petal_length,5.9']
When value is single
['sepal_length,6.8', 'sepal_width,3.2', 'petal_length,5.9', 'petal_width,2.3']
final_union:  []
Final Union Set:  []
When item is found as key in the block
{144, 113, 77}
When item is found as key in the block
{144}
When item is found as key in the block
{144}
When item is found as key in the block
{144}
Final tmp_set:  {144}
This is the final value:  {144}
K_144


When value is single
['sepal_length,6.7']
When value is single
['sepal_length,6.7', 'sepal_width,3.3']
When value is single
['sepal_length,6.7', 'sepal_width,3.3', 'petal_length,5.7']
When value is single
['sepal_length,6.7', 'sepal_width,3.3', 'petal_length,5.7', 'petal_width,2.5']
final_union:  []
Final Union Set:  []
When item is found as 

In [40]:
dic

{'K_1': {1},
 'K_2': {13, 46},
 'K_3': {3},
 'K_4': {4},
 'K_5': {50},
 'K_6': {6},
 'K_7': {7},
 'K_8': {8},
 'K_9': set(),
 'K_10': {35, 38},
 'K_11': {11},
 'K_12': {25},
 'K_13': {13},
 'K_14': {14},
 'K_15': {15},
 'K_16': {16},
 'K_17': {17},
 'K_18': {18},
 'K_19': set(),
 'K_20': {20},
 'K_21': {21},
 'K_22': {22},
 'K_23': {23},
 'K_24': {24},
 'K_25': {25},
 'K_26': {26},
 'K_27': set(),
 'K_28': set(),
 'K_29': {29},
 'K_30': {30},
 'K_31': {31},
 'K_32': set(),
 'K_33': {33},
 'K_34': {34},
 'K_35': {35, 38},
 'K_36': set(),
 'K_37': {37},
 'K_38': {35, 38},
 'K_39': set(),
 'K_40': set(),
 'K_41': {41},
 'K_42': {42},
 'K_43': {3},
 'K_44': {44},
 'K_45': set(),
 'K_46': {46},
 'K_47': set(),
 'K_48': {48},
 'K_49': set(),
 'K_50': {50},
 'K_51': set(),
 'K_52': set(),
 'K_53': {53},
 'K_54': {90},
 'K_55': set(),
 'K_56': {56},
 'K_57': set(),
 'K_58': {58},
 'K_59': {59},
 'K_60': set(),
 'K_61': set(),
 'K_62': {62},
 'K_63': {63},
 'K_64': {64},
 'K_65': {65},
 'K_66':

#### Lower Approximation !!

In [41]:
def lowerApproximation(charac_set,concept):
    #set to contain lower approximations
    lower = set()
    
    #Check for each element of the concept
    for item in concept:
        key = 'K_%d' % (item)
        set_value = charac_set[key]
  
        if set_value.issubset(set(concept)):
            lower = lower.union(set_value)
          
    return lower

In [42]:
lower_approximations = {}
for item in goal_list:
    #Key is the string converted list so as to add as dictionary key
    lower_approximations[str(item)] = lowerApproximation(dic,item)

In [43]:
lower_goal_list=list(lower_approximations.values())
print(lower_goal_list)

[{1, 3, 4, 6, 7, 8, 11, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 29, 30, 31, 33, 34, 35, 37, 38, 41, 42, 44, 46, 48, 50}, {64, 65, 66, 68, 69, 71, 72, 73, 75, 79, 81, 82, 84, 86, 87, 89, 90, 92, 93, 94, 97, 98, 99, 53, 56, 58, 59, 62, 63}, {128, 129, 130, 131, 132, 135, 136, 137, 138, 139, 140, 143, 144, 145, 146, 147, 148, 149, 150, 101, 102, 104, 106, 107, 108, 109, 111, 113, 118, 119, 120, 121, 122, 123, 124, 127}]


#### Upper Approximation !!

In [44]:
def upperApproximation(charac_set,concept):
    #set to contain lower approximations
    upper = set()
    
    #Check for each element of the concept
    for item in concept:
        key = 'K_%d' % (item)
        set_value = charac_set[key]
        upper = upper.union(set_value)
          
    return upper

In [45]:
upper_approximations = {}
for item in goal_list:
    #Key is the string converted list so as to add as dictionary key
    upper_approximations[str(item)] = upperApproximation(dic,item)

In [46]:
upper_goal_list=list(upper_approximations.values())
print(upper_goal_list)

[{1, 3, 4, 6, 7, 8, 11, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 29, 30, 31, 33, 34, 35, 37, 38, 41, 42, 44, 46, 48, 50}, {64, 65, 66, 68, 69, 71, 72, 73, 75, 79, 81, 82, 84, 86, 87, 89, 90, 92, 93, 94, 97, 98, 99, 53, 56, 58, 59, 62, 63}, {128, 129, 130, 131, 132, 135, 136, 137, 138, 139, 140, 143, 144, 145, 146, 147, 148, 149, 150, 55, 73, 101, 102, 104, 105, 106, 107, 108, 109, 111, 113, 118, 119, 120, 121, 122, 123, 124, 127}]


#### Middle Approximation !!

In [47]:
first_column = list(dic.keys())

In [48]:
second_column = list(dic.values())

In [49]:
prob_approx = pd.DataFrame(
    {'charset_name': first_column,
     'charset_value': second_column
    })

In [50]:
prob_approx

,charset_name,charset_value
0,K_1,{1}
1,K_2,"{13, 46}"
2,K_3,{3}
3,K_4,{4}
4,K_5,{50}
5,K_6,{6}
6,K_7,{7}
7,K_8,{8}
8,K_9,{}
9,K_10,"{35, 38}"


In [51]:
def probabilisticApproximation(concept):
    prob = []
    for index, row in prob_approx.iterrows():
        if len(row['charset_value']) != 0:
            probability_conditional = len(row['charset_value'].intersection(set(concept))) / len(row['charset_value'])
        else:
            probability_conditional = 0.0
            
        prob.append(round(probability_conditional,2))
    return prob

In [52]:
def findMiddleApprox(concept):
    probapprox = set()
    cond_prob = probabilisticApproximation(concept) #Need to put the goal
    prob_approx['cond_probability'] = cond_prob
    print(prob_approx)
    for index, row in prob_approx.iterrows():
        part1,part2 = row['charset_name'].split("_")
       
        if row['cond_probability'] >= 0.50:
            if int(part2) in concept:
#                 print(row['charset_value'])
                probapprox = probapprox.union(row['charset_value'])
    
    return probapprox  

In [53]:
middle_approximations = {}
for item in goal_list:
    print(item)
    #Key is the string converted list so as to add as dictionary key
    middle_approximations[str(item)] = findMiddleApprox(item)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]
    charset_name        charset_value  cond_probability
0            K_1                  {1}               1.0
1            K_2             {13, 46}               1.0
2            K_3                  {3}               1.0
3            K_4                  {4}               1.0
4            K_5                 {50}               1.0
5            K_6                  {6}               1.0
6            K_7                  {7}               1.0
7            K_8                  {8}               1.0
8            K_9                   {}               0.0
9           K_10             {35, 38}               1.0
10          K_11                 {11}               1.0
11          K_12                 {25}               1.0
12          K_13                 {13}               1.0
13          K_14        

    charset_name        charset_value  cond_probability
0            K_1                  {1}              0.00
1            K_2             {13, 46}              0.00
2            K_3                  {3}              0.00
3            K_4                  {4}              0.00
4            K_5                 {50}              0.00
5            K_6                  {6}              0.00
6            K_7                  {7}              0.00
7            K_8                  {8}              0.00
8            K_9                   {}              0.00
9           K_10             {35, 38}              0.00
10          K_11                 {11}              0.00
11          K_12                 {25}              0.00
12          K_13                 {13}              0.00
13          K_14                 {14}              0.00
14          K_15                 {15}              0.00
15          K_16                 {16}              0.00
16          K_17                 {17}           

In [54]:
middle_goal_list=list(middle_approximations.values())
print(middle_goal_list)

[{1, 3, 4, 6, 7, 8, 11, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 29, 30, 31, 33, 34, 35, 37, 38, 41, 42, 44, 46, 48, 50}, {64, 65, 66, 68, 69, 71, 72, 73, 75, 79, 81, 82, 84, 86, 87, 89, 90, 92, 93, 94, 97, 98, 99, 53, 56, 58, 59, 62, 63}, {128, 129, 130, 131, 132, 135, 136, 137, 138, 139, 140, 143, 144, 145, 146, 147, 148, 149, 150, 55, 101, 102, 104, 105, 106, 107, 108, 109, 111, 113, 118, 119, 120, 121, 122, 123, 124, 127}]


In [55]:
df2

,Cases,att_val
0,"sepal_length,4.3..4.3","[43, 2, 19, 14]"
1,"sepal_length,4.3..7.9","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,"sepal_length,4.3..4.5","[2, 39, 9, 42, 43, 14, 19]"
3,"sepal_length,4.5..7.9","[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 15, 1..."
4,"sepal_length,4.3..4.5","[2, 39, 9, 42, 43, 14, 19]"
5,"sepal_length,4.5..7.9","[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 15, 1..."
6,"sepal_length,4.3..4.7","[2, 3, 4, 7, 39, 9, 42, 43, 14, 48, 19, 23, 30]"
7,"sepal_length,4.7..7.9","[1, 2, 3, 5, 6, 8, 10, 11, 12, 13, 15, 16, 17,..."
8,"sepal_length,4.3..4.8","[2, 3, 4, 7, 39, 9, 42, 43, 12, 13, 14, 46, 48..."
9,"sepal_length,4.8..7.9","[1, 2, 5, 6, 8, 10, 11, 12, 13, 15, 16, 17, 18..."


In [56]:
df3=df2.copy()

In [57]:
import pylab as pl

In [58]:
def findGoalIntersect(goal):
    goalIntersect = []
    
    for index, row in df3.iterrows():
        #List containing intersection of (a,v) pairs and goal
        goalIntersect.append(set(row['att_val']).intersection(set(goal)))
          
    #Check if goal_intersect column exists
    if 'goal_intersect' in df3:
        df3['goal_intersect'] = goalIntersect
    else:
        #Insert new column with the recent iteration
        df3.insert(2, 'goal_intersect', goalIntersect)

In [59]:
def findCases(df3):
    case_to_be = []
    #Find the cases with maximum goal coverage
    m = max(df3['goal_intersect'], key=len)
    possible_cases = [i for i, j in enumerate(df3['goal_intersect'].tolist()) if j == m]
    
    #Index of the case covering max goal and having min no. of elements
    new_df = df3.iloc[possible_cases,:]
    
    m1 = min(new_df['att_val'], key=len)
    
    for index,row in new_df.iterrows():
        if row['att_val'] == m1:
            case_to_be.append(index)
   
    return case_to_be[0]

In [60]:
def combineInterval(test_condition):
    
    test_num = [] #This will contain the conditions having interval
    test_str = [] #This will contain the conditions having no interval
    
    #Loop through to seprate contions having intervals and no intervals
    for item in test_condition:
        if ".." in item:
            test_num.append(item)
        else:
            test_str.append(item)
   
    #Group the conditions having interval based on same attributes
    grouped = [list(g) for k, g in groupby(test_num, lambda s: s.partition(',')[0])]
    
    final_list = []
    
    #Actually combining the intervals
    for list1 in grouped:
        greatest = 0
        smallest = 0
        for item in list1:
            part1,part2 = item.split(",")
            start,stop = part2.split("..")
            start = float(start)
            stop = float(stop)

            if greatest == 0 and smallest == 0:
                greatest = start
                smallest = stop

            if start > greatest:
                greatest = float(start)

            if stop < smallest:
                smallest = float(stop)

        con_tmp = part1 + "," + str(greatest) + ".." + str(smallest)
        final_list.append(con_tmp)  
            
    actual_condition = final_list + test_str
    
    return actual_condition

In [61]:
def dropCondition(condition,current_goal):
    
    for item in range(0,len(condition)):
        temp_att_val = []
        temp_cond = condition.copy() #use list.copy() as equal operator simply copies over the reference
        temp_cond.remove(condition[item])
       
        #temp_cond contains the elements after removing the current element
        for i in temp_cond:
            if i is not None:
                location = df3.index[df3['Cases'] == i].tolist()
                element = df3['att_val'].loc[location[0]]
                temp_att_val.append(set(element))
          
        #temp_att_val contains the actual value set of the corresponding cases
        #Find the intersection if the set has more than one element, otherwise no need
        if len(temp_att_val) > 1:
            intersection = set.intersection(*temp_att_val)
        
            #if the set still remains a subset of the original goal after removing current element
            #then set the current element to None as we want to drop this later
            if intersection.issubset(current_goal):
                condition[item] = None
    
    condition = [x for x in condition if x is not None]
   
    return condition   

In [62]:
def stepAlgo(df3,selected_case,current_goal,B,condition,concept_curr):
    
    rule_set = []
    original_goal = current_goal.copy()
    
    
    while current_goal != set():
        #Check if the selected case is a subset of the current goal
        #List of current case
        A = df3['att_val'].loc[selected_case] 

        if B == set():
            #Copy over the current set elements to B
            for i in range(len(A)):
                B.add(A[i])

        #Elements of intersection of current and previous set
        A = set(A).intersection(B)
        B = A.copy()
        
        print("Intersection set : ",A)
        #Check if intersecting elements are subset of Goal
        if A.issubset(current_goal):
          
            #Current goal is updated after discarding the already covered goal by new rule
            if len(A) != 0:
                current_goal = set(current_goal) - A
            else:
                current_goal = set()
            

            #Extract the current case
            curr_case = df3['Cases'].loc[selected_case]
            #Add the conditions of a Rule
            condition.append(curr_case)
            
            #Check for possibility of dropping conditions
            if len(condition) > 1:
                condition = dropCondition(condition,original_goal)
            
            #Combine the interval
            if len(condition) > 1:
                condition = combineInterval(condition)
            

            #Join conditions
            cond = ""
            for item in condition:
                cond = cond + "(" + str(item) + ")" + " & "

            cond = cond[:-2] + "->"
            rule = cond + " (" + concept + "," + concept_curr + ")"
            rule_set.append(rule)
            
            #Reset everythng and continue for covering rest of the goal
            condition = []
            B = set()
            findGoalIntersect(current_goal)
            selected_case = findCases(df3)


        #If not a subset of current goal
        else:
            print("NOT")
            #Assign empty set for the selected case for next iteration
            df3['goal_intersect'].loc[selected_case] = set()
            print(df3['Cases'].loc[selected_case], " Set to be NULL")
           
            #Extract the current case
            curr_case = df3['Cases'].loc[selected_case]
            #Add the case to the condition list
            condition.append(curr_case)
            
            #Check for Range overlapping of the remaining cases
            if ".." in curr_case:
                second_part = curr_case.split(',')[1]
                start = float(second_part.split('..')[0])
                end = float(second_part.split('..')[1])
                for index, row in df3.iterrows():
                    if ".." in row['Cases']:
                        part2 = row['Cases'].split(',')[1]
                        start1 = float(part2.split('..')[0])
                        end1 = float(part2.split('..')[1])

                        #Assign blank set for cases with overlapping ranges
                        if set((pl.frange(start,end))).issubset(pl.frange(start1,end1)) == True:
                            if start == start1 and end <= end1:
                                row['goal_intersect'] = set()
                                print(row['Cases'], "also set to be NULL")
            
           
            selected_case = findCases(df3)
           
                        
    return rule_set

In [63]:
#concept_list and goal_list has 1:1 mapping
final_rules = []
start_time = time.time()

#Running algorithm for all the goals - Lower Approximation/concepts
for i in range(0,len(middle_goal_list)):
    findGoalIntersect(list(middle_goal_list[i]))
    condition = []
    B = set()
    selected_case = findCases(df3)
    
    rule_set = stepAlgo(df3,selected_case,middle_goal_list[i],B,condition,concept_list[i])
    final_rules.append(rule_set)
    
elapsed_time = time.time() - start_time

Intersection set :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50}
NOT
petal_length,1.0..2.5  Set to be NULL
petal_length,1.0..2.5 also set to be NULL
petal_length,1.0..3.1 also set to be NULL
petal_length,1.0..3.4 also set to be NULL
petal_length,1.0..3.5 also set to be NULL
petal_length,1.0..3.7 also set to be NULL
petal_length,1.0..3.8 also set to be NULL
petal_length,1.0..3.8 also set to be NULL
petal_length,1.0..4.0 also set to be NULL
petal_length,1.0..4.0 also set to be NULL
petal_length,1.0..4.2 also set to be NULL
petal_length,1.0..4.2 also set to be NULL
petal_length,1.0..4.3 also set to be NULL
petal_length,1.0..4.5 also set to be NULL
petal_length,1.0..4.5 also set to be NULL
petal_length,1.0..4.7 also set to be NULL
petal_length,1.0..4.8 also set to be NULL
petal_length,1.0..4.8 also set to be NULL
petal_length,1.0..5.0 also set t

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:86: MatplotlibDeprecationWarning: numpy.arange


 also set to be NULL
Intersection set :  {1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50}
NOT
sepal_length,4.5..7.9  Set to be NULL
sepal_length,4.5..7.9 also set to be NULL
sepal_length,4.5..7.9 also set to be NULL
Intersection set :  {1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50}
NOT
sepal_length,4.3..5.7  Set to be NULL
sepal_length,4.3..7.9 also set to be NULL
sepal_length,4.3..5.7 also set to be NULL
sepal_length,4.3..5.8 also set to be NULL
sepal_length,4.3..5.8 also set to be NULL
sepal_length,4.3..6.0 also set to be NULL
sepal_length,4.3..6.0 also set to be NULL
sepal_length,4.3..6.2 also set to be NULL
sepal_length,4.3..6.2 also set to be NULL
sepal_length,4.3..6.3 also set to be NULL
sepal_length,4.3..6.5 also set to b

sepal_width,2.2..4.4 also set to be NULL
sepal_width,2.2..3.8 also set to be NULL
sepal_width,2.2..3.8 also set to be NULL
sepal_width,2.2..4.0 also set to be NULL
sepal_width,2.2..4.0 also set to be NULL
sepal_width,2.2..4.2 also set to be NULL
sepal_width,2.2..4.3 also set to be NULL
Intersection set :  {1, 2, 3, 5, 8, 11, 12, 18, 20, 26, 27, 28, 29, 30, 31, 32, 36, 37, 40, 41, 43, 46, 47, 49, 50}
NOT
petal_width,0.1..0.3  Set to be NULL
petal_width,0.1..0.3 also set to be NULL
petal_width,0.1..0.5 also set to be NULL
petal_width,0.1..0.6 also set to be NULL
petal_width,0.1..0.8 also set to be NULL
petal_width,0.1..1.1 also set to be NULL
petal_width,0.1..1.1 also set to be NULL
petal_width,0.1..1.2 also set to be NULL
petal_width,0.1..1.4 also set to be NULL
petal_width,0.1..1.4 also set to be NULL
petal_width,0.1..1.6 also set to be NULL
petal_width,0.1..1.6 also set to be NULL
petal_width,0.1..1.8 also set to be NULL
petal_width,0.1..1.9 also set to be NULL
petal_width,0.1..1.9 al

Intersection set :  {51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150}
NOT
petal_length,2.5..6.9  Set to be NULL
petal_length,2.5..6.9 also set to be NULL
Intersection set :  {51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 14

...

sepal_width,2.2..4.2 also set to be NULL
sepal_width,2.2..4.3 also set to be NULL
Intersection set :  {134, 54, 56, 57, 62, 64, 67, 68, 70, 72, 74, 79, 80, 81, 83, 88, 89, 90, 91, 92, 93, 95, 96, 97, 98, 100}
NOT
petal_length,3.8..6.9  Set to be NULL
petal_length,3.8..6.9 also set to be NULL
petal_length,3.8..6.9 also set to be NULL
Intersection set :  {134, 54, 56, 57, 62, 64, 67, 70, 72, 74, 79, 81, 83, 88, 89, 90, 91, 92, 93, 95, 96, 97, 98, 100}
NOT
petal_width,1.1..2.5  Set to be NULL
petal_length,1.1..6.9 also set to be NULL
petal_length,1.1..6.9 also set to be NULL
petal_width,1.1..2.5 also set to be NULL
petal_width,1.1..2.5 also set to be NULL
Intersection set :  {54, 56, 62, 64, 67, 70, 72, 74, 79, 81, 83, 89, 90, 91, 92, 93, 95, 96, 97, 98, 100}
NOT
sepal_length,4.3..6.2  Set to be NULL
sepal_length,4.3..7.9 also set to be NULL
sepal_length,4.3..6.2 also set to be NULL
sepal_length,4.3..6.2 also set to be NULL
sepal_length,4.3..6.3 also set to be NULL
sepal_length,4.3..6.5 a

Intersection set :  {128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 51, 52, 53, 55, 56, 57, 59, 64, 67, 69, 71, 73, 74, 77, 78, 79, 80, 84, 85, 86, 87, 92, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127}
NOT
petal_length,4.5..6.9  Set to be NULL
sepal_length,4.5..7.9 also set to be NULL
sepal_length,4.5..7.9 also set to be NULL
petal_length,4.5..6.9 also set to be NULL
petal_length,4.5..6.9 also set to be NULL
Intersection set :  {128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 51, 52, 53, 55, 57, 64, 67, 69, 71, 73, 77, 78, 79, 84, 85, 86, 87, 92, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127}
NOT
petal_width,1.4..2.5  Set to be NULL
petal_length,1.4..6.9 also set to be NULL
petal

Intersection set :  {128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 51, 52, 53, 55, 57, 64, 67, 69, 71, 73, 77, 78, 79, 84, 85, 86, 87, 92, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127}
NOT
sepal_length,4.7..7.9  Set to be NULL
sepal_length,4.7..7.9 also set to be NULL
Intersection set :  {128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 51, 52, 53, 55, 57, 64, 67, 69, 71, 73, 77, 78, 79, 84, 85, 86, 87, 92, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127}
NOT
sepal_width,2.2..3.8  Set to be NULL
sepal_width,2.2..4.4 also set to be NULL
sepal_width,2.2..3.8 also set to be NULL
sepal_width,2.2..3.8 also set to be NULL
sepal_width,2.2..4.0 also set to be NULL
sepal_width,2.2..4.0 also

petal_width,0.1..2.3 also set to be NULL
petal_width,0.1..2.5 also set to be NULL
Intersection set :  {128, 129, 130, 131, 133, 136, 138, 139, 140, 141, 142, 143, 144, 146, 147, 148, 150, 71, 78, 84, 102, 103, 104, 105, 106, 108, 109, 111, 112, 113, 114, 116, 117, 121, 123, 124, 125, 126, 127}
NOT
sepal_width,2.2..3.3  Set to be NULL
sepal_width,2.2..4.4 also set to be NULL
sepal_width,2.2..3.3 also set to be NULL
sepal_width,2.2..3.5 also set to be NULL
sepal_width,2.2..3.5 also set to be NULL
sepal_width,2.2..3.7 also set to be NULL
sepal_width,2.2..3.8 also set to be NULL
sepal_width,2.2..3.8 also set to be NULL
sepal_width,2.2..4.0 also set to be NULL
sepal_width,2.2..4.0 also set to be NULL
sepal_width,2.2..4.2 also set to be NULL
sepal_width,2.2..4.3 also set to be NULL
Intersection set :  {128, 129, 130, 131, 133, 136, 138, 139, 140, 141, 142, 143, 144, 146, 147, 148, 150, 71, 78, 84, 102, 103, 104, 105, 108, 109, 111, 112, 113, 114, 116, 117, 121, 124, 125, 126, 127}
NOT
petal_

sepal_length,4.3..7.0 also set to be NULL
sepal_length,4.3..7.0 also set to be NULL
sepal_length,4.3..7.2 also set to be NULL
sepal_length,4.3..7.2 also set to be NULL
sepal_length,4.3..7.3 also set to be NULL
sepal_length,4.3..7.5 also set to be NULL
sepal_length,4.3..7.7 also set to be NULL
sepal_length,4.3..7.8 also set to be NULL
petal_length,4.3..6.9 also set to be NULL
Intersection set :  {112, 117}
NOT
sepal_width,3.0..4.4  Set to be NULL
sepal_width,3.0..4.4 also set to be NULL
sepal_width,3.0..4.4 also set to be NULL
Intersection set :  {117}
NOT
petal_length,5.5..6.9  Set to be NULL
sepal_length,5.5..7.9 also set to be NULL
sepal_length,5.5..7.9 also set to be NULL
petal_length,5.5..6.9 also set to be NULL
petal_length,5.5..6.9 also set to be NULL
Intersection set :  {117}
NOT
sepal_length,6.5..7.9  Set to be NULL
sepal_length,6.5..7.9 also set to be NULL
sepal_length,6.5..7.9 also set to be NULL
Intersection set :  {117}
NOT
petal_length,1.0..5.5  Set to be NULL
petal_length

In [64]:
print("Time to run the algorithm (for Middle Approximation): ", round(elapsed_time, 3), "Sec")
print("\n")
print(*final_rules, sep='\n')

with open('../data/Iris/Rules/Iris_15_attcon_middle.txt', 'w') as f:
    for item in final_rules:
        f.write("%s\n" % item)

Time to run the algorithm (for Middle Approximation):  9.104 Sec


['(sepal_width,3.5..3.3) & (sepal_length,4.3..4.8) -> (class,Iris-setosa)']
['(sepal_length,6.0..7.9) & (petal_length,1.0..4.2) & (sepal_width,2.2..2.7) & (petal_length,4.5..6.9) -> (class,Iris-versicolor)']
['(petal_width,2.0..1.9) & (sepal_length,4.3..6.3) -> (class,Iris-viginica)']


In [65]:
#concept_list and goal_list has 1:1 mapping
final_rules = []
start_time = time.time()

#Running algorithm for all the goals - Lower Approximation/concepts
for i in range(0,len(lower_goal_list)):
    findGoalIntersect(list(lower_goal_list[i]))
    condition = []
    B = set()
    selected_case = findCases(df3)
    
    rule_set = stepAlgo(df3,selected_case,lower_goal_list[i],B,condition,concept_list[i])
    final_rules.append(rule_set)
    
elapsed_time = time.time() - start_time

Intersection set :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50}
NOT
petal_length,1.0..2.5  Set to be NULL
petal_length,1.0..2.5 also set to be NULL
petal_length,1.0..3.1 also set to be NULL
petal_length,1.0..3.4 also set to be NULL
petal_length,1.0..3.5 also set to be NULL
petal_length,1.0..3.7 also set to be NULL
petal_length,1.0..3.8 also set to be NULL
petal_length,1.0..3.8 also set to be NULL
petal_length,1.0..4.0 also set to be NULL
petal_length,1.0..4.0 also set to be NULL
petal_length,1.0..4.2 also set to be NULL
petal_length,1.0..4.2 also set to be NULL
petal_length,1.0..4.3 also set to be NULL
petal_length,1.0..4.5 also set to be NULL
petal_length,1.0..4.5 also set to be NULL
petal_length,1.0..4.7 also set to be NULL
petal_length,1.0..4.8 also set to be NULL
petal_length,1.0..4.8 also set to be NULL
petal_length,1.0..5.0 also set t

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:86: MatplotlibDeprecationWarning: numpy.arange


 also set to be NULL
sepal_width,3.0..4.4 also set to be NULL
Intersection set :  {1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 40, 41, 43, 44, 45, 46, 47, 48, 49, 50}
NOT
sepal_width,2.8..4.4  Set to be NULL
sepal_width,2.8..4.4 also set to be NULL
sepal_width,2.8..4.4 also set to be NULL
Intersection set :  {1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 40, 41, 43, 44, 45, 46, 47, 48, 49, 50}
NOT
sepal_width,2.7..4.4  Set to be NULL
sepal_width,2.7..4.4 also set to be NULL
Intersection set :  {1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 40, 41, 43, 44, 45, 46, 47, 48, 49, 50}
NOT
sepal_width,2.5..4.4  Set to be NULL
sepal_width,2.5..4.4 also set to be NULL
sepal_width,2.5..4.4 also set to be NULL
petal_length,2.5..6.9 also set 

Intersection set :  {1, 2, 3, 5, 8, 11, 12, 18, 20, 22, 26, 27, 28, 29, 30, 31, 32, 36, 37, 40, 41, 43, 46, 47, 49, 50}
NOT
sepal_width,2.2..3.8  Set to be NULL
sepal_width,2.2..4.4 also set to be NULL
sepal_width,2.2..3.8 also set to be NULL
sepal_width,2.2..3.8 also set to be NULL
sepal_width,2.2..4.0 also set to be NULL
sepal_width,2.2..4.0 also set to be NULL
sepal_width,2.2..4.2 also set to be NULL
sepal_width,2.2..4.3 also set to be NULL
Intersection set :  {1, 2, 3, 5, 8, 11, 12, 18, 20, 26, 27, 28, 29, 30, 31, 32, 36, 37, 40, 41, 43, 46, 47, 49, 50}
NOT
petal_width,0.1..0.3  Set to be NULL
petal_width,0.1..0.3 also set to be NULL
petal_width,0.1..0.5 also set to be NULL
petal_width,0.1..0.6 also set to be NULL
petal_width,0.1..0.8 also set to be NULL
petal_width,0.1..1.1 also set to be NULL
petal_width,0.1..1.1 also set to be NULL
petal_width,0.1..1.2 also set to be NULL
petal_width,0.1..1.4 also set to be NULL
petal_width,0.1..1.4 also set to be NULL
petal_width,0.1..1.6 also 

Intersection set :  {51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150}
NOT
petal_length,2.5..6.9  Set to be NULL
petal_length,2.5..6.9 also set to be NULL
Intersection set :  {51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 14

petal_width,0.1..2.2 also set to be NULL
petal_width,0.1..2.2 also set to be NULL
petal_width,0.1..2.3 also set to be NULL
petal_width,0.1..2.5 also set to be NULL
Intersection set :  {134, 52, 54, 55, 56, 57, 59, 60, 61, 62, 64, 65, 66, 67, 68, 70, 72, 73, 74, 75, 76, 78, 79, 80, 81, 82, 83, 85, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 100, 117}
NOT
sepal_width,2.3..4.4  Set to be NULL
sepal_width,2.3..4.4 also set to be NULL
Intersection set :  {134, 52, 54, 55, 56, 57, 59, 60, 61, 62, 64, 65, 66, 67, 68, 70, 72, 73, 74, 75, 76, 78, 79, 80, 81, 82, 83, 85, 87, 88, 89, 90, 91, 92, 93, 95, 96, 97, 98, 100, 117}
NOT
petal_length,3.4..6.9  Set to be NULL
petal_length,3.4..6.9 also set to be NULL
Intersection set :  {134, 52, 54, 55, 56, 57, 59, 60, 61, 62, 64, 65, 66, 67, 68, 70, 72, 73, 74, 75, 76, 78, 79, 80, 81, 82, 83, 85, 87, 88, 89, 90, 91, 92, 93, 95, 96, 97, 98, 100, 117}
NOT
petal_length,3.5..6.9  Set to be NULL
petal_length,3.5..6.9 also set to be NULL
Intersection set :

petal_length,1.0..5.7 also set to be NULL
petal_length,1.0..5.8 also set to be NULL
petal_length,1.0..5.8 also set to be NULL
petal_length,1.0..6.0 also set to be NULL
petal_length,1.0..6.0 also set to be NULL
petal_length,1.0..6.2 also set to be NULL
petal_length,1.0..6.3 also set to be NULL
petal_length,1.0..6.5 also set to be NULL
petal_length,1.0..6.7 also set to be NULL
petal_length,1.0..6.8 also set to be NULL
Intersection set :  {96, 97, 98, 100, 72, 56, 93}
NOT
petal_width,0.1..1.4  Set to be NULL
petal_width,0.1..1.4 also set to be NULL
petal_width,0.1..1.4 also set to be NULL
petal_width,0.1..1.6 also set to be NULL
petal_width,0.1..1.6 also set to be NULL
petal_width,0.1..1.8 also set to be NULL
petal_width,0.1..1.9 also set to be NULL
petal_width,0.1..1.9 also set to be NULL
petal_width,0.1..2.0 also set to be NULL
petal_width,0.1..2.2 also set to be NULL
petal_width,0.1..2.2 also set to be NULL
petal_width,0.1..2.3 also set to be NULL
petal_width,0.1..2.5 also set to be NU

Intersection set :  {128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 51, 52, 53, 55, 57, 64, 67, 69, 71, 73, 77, 78, 79, 84, 85, 86, 87, 92, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127}
NOT
petal_length,3.8..6.9  Set to be NULL
petal_length,3.8..6.9 also set to be NULL
petal_length,3.8..6.9 also set to be NULL
Intersection set :  {128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 51, 52, 53, 55, 57, 64, 67, 69, 71, 73, 77, 78, 79, 84, 85, 86, 87, 92, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127}
NOT
petal_length,3.7..6.9  Set to be NULL
petal_length,3.7..6.9 also set to be NULL
Intersection set :  {128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144

Intersection set :  {128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 53, 71, 73, 77, 78, 84, 101, 102, 103, 104, 105, 106, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 121, 122, 123, 124, 125, 126, 127}
NOT
petal_length,1.0..6.7  Set to be NULL
petal_length,1.0..6.7 also set to be NULL
petal_length,1.0..6.8 also set to be NULL
Intersection set :  {128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 53, 71, 73, 77, 78, 84, 101, 102, 103, 104, 105, 106, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 121, 123, 124, 125, 126, 127}
NOT
sepal_length,5.8..7.9  Set to be NULL
sepal_length,5.8..7.9 also set to be NULL
sepal_length,5.8..7.9 also set to be NULL
Intersection set :  {128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 53, 71, 73, 77, 78, 84, 101, 102, 103, 104, 105, 106, 108

sepal_length,4.3..7.9 also set to be NULL
sepal_length,4.3..6.8 also set to be NULL
sepal_length,4.3..6.8 also set to be NULL
sepal_length,4.3..7.0 also set to be NULL
sepal_length,4.3..7.0 also set to be NULL
sepal_length,4.3..7.2 also set to be NULL
sepal_length,4.3..7.2 also set to be NULL
sepal_length,4.3..7.3 also set to be NULL
sepal_length,4.3..7.5 also set to be NULL
sepal_length,4.3..7.7 also set to be NULL
sepal_length,4.3..7.8 also set to be NULL
petal_length,4.3..6.9 also set to be NULL
Intersection set :  {129, 133, 104, 105, 78, 112, 113, 148, 117, 125}
NOT
sepal_width,2.2..3.0  Set to be NULL
sepal_width,2.2..4.4 also set to be NULL
sepal_width,2.2..3.0 also set to be NULL
sepal_width,2.2..3.0 also set to be NULL
sepal_width,2.2..3.2 also set to be NULL
sepal_width,2.2..3.2 also set to be NULL
sepal_width,2.2..3.3 also set to be NULL
sepal_width,2.2..3.5 also set to be NULL
sepal_width,2.2..3.5 also set to be NULL
sepal_width,2.2..3.7 also set to be NULL
sepal_width,2.2.

In [66]:
print("Time to run the algorithm (for Lower Approximation): ", round(elapsed_time, 3), "Sec")
print("\n")
print(*final_rules, sep='\n')

with open('../data/Iris/Rules/Iris_15_attcon_lower.txt', 'w') as f:
    for item in final_rules:
        f.write("%s\n" % item)

Time to run the algorithm (for Lower Approximation):  9.172 Sec


['(sepal_width,3.5..3.3) & (sepal_length,4.3..4.8) -> (class,Iris-setosa)']
['(sepal_length,6.0..7.9) & (petal_length,1.0..4.2) & (sepal_width,2.2..2.7) & (petal_length,4.5..6.9) -> (class,Iris-versicolor)']
['(petal_width,2.0..1.9) & (sepal_length,4.3..6.3) -> (class,Iris-viginica)']


In [67]:
#concept_list and goal_list has 1:1 mapping
final_rules = []
start_time = time.time()

#Running algorithm for all the goals - Lower Approximation/concepts
for i in range(0,len(upper_goal_list)):
    findGoalIntersect(list(upper_goal_list[i]))
    condition = []
    B = set()
    selected_case = findCases(df3)
    
    rule_set = stepAlgo(df3,selected_case,upper_goal_list[i],B,condition,concept_list[i])
    final_rules.append(rule_set)
    
elapsed_time = time.time() - start_time

Intersection set :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50}
NOT
petal_length,1.0..2.5  Set to be NULL
petal_length,1.0..2.5 also set to be NULL
petal_length,1.0..3.1 also set to be NULL
petal_length,1.0..3.4 also set to be NULL
petal_length,1.0..3.5 also set to be NULL
petal_length,1.0..3.7 also set to be NULL
petal_length,1.0..3.8 also set to be NULL
petal_length,1.0..3.8 also set to be NULL
petal_length,1.0..4.0 also set to be NULL
petal_length,1.0..4.0 also set to be NULL
petal_length,1.0..4.2 also set to be NULL
petal_length,1.0..4.2 also set to be NULL
petal_length,1.0..4.3 also set to be NULL
petal_length,1.0..4.5 also set to be NULL
petal_length,1.0..4.5 also set to be NULL
petal_length,1.0..4.7 also set to be NULL
petal_length,1.0..4.8 also set to be NULL
petal_length,1.0..4.8 also set to be NULL
petal_length,1.0..5.0 also set t

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:86: MatplotlibDeprecationWarning: numpy.arange


Intersection set :  {1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50}
NOT
sepal_length,4.5..7.9  Set to be NULL
sepal_length,4.5..7.9 also set to be NULL
sepal_length,4.5..7.9 also set to be NULL
Intersection set :  {1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50}
NOT
sepal_length,4.3..5.7  Set to be NULL
sepal_length,4.3..7.9 also set to be NULL
sepal_length,4.3..5.7 also set to be NULL
sepal_length,4.3..5.8 also set to be NULL
sepal_length,4.3..5.8 also set to be NULL
sepal_length,4.3..6.0 also set to be NULL
sepal_length,4.3..6.0 also set to be NULL
sepal_length,4.3..6.2 also set to be NULL
sepal_length,4.3..6.2 also set to be NULL
sepal_length,4.3..6.3 also set to be NULL
sepal_length,4.3..6.5 also set to be NULL
sepal_length,4

petal_width,0.2..2.5 also set to be NULL
petal_width,0.2..2.5 also set to be NULL
Intersection set :  {1, 2, 3, 5, 8, 11, 12, 17, 18, 20, 22, 26, 27, 28, 29, 30, 31, 32, 36, 37, 40, 41, 43, 46, 47, 49, 50}
NOT
sepal_length,4.7..7.9  Set to be NULL
sepal_length,4.7..7.9 also set to be NULL
Intersection set :  {1, 2, 3, 5, 8, 11, 12, 18, 20, 22, 26, 27, 28, 29, 30, 31, 32, 36, 37, 40, 41, 43, 46, 47, 49, 50}
NOT
sepal_width,2.2..3.8  Set to be NULL
sepal_width,2.2..4.4 also set to be NULL
sepal_width,2.2..3.8 also set to be NULL
sepal_width,2.2..3.8 also set to be NULL
sepal_width,2.2..4.0 also set to be NULL
sepal_width,2.2..4.0 also set to be NULL
sepal_width,2.2..4.2 also set to be NULL
sepal_width,2.2..4.3 also set to be NULL
Intersection set :  {1, 2, 3, 5, 8, 11, 12, 18, 20, 26, 27, 28, 29, 30, 31, 32, 36, 37, 40, 41, 43, 46, 47, 49, 50}
NOT
petal_width,0.1..0.3  Set to be NULL
petal_width,0.1..0.3 also set to be NULL
petal_width,0.1..0.5 also set to be NULL
petal_width,0.1..0.6 al

Intersection set :  set()
Intersection set :  {51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150}
NOT
petal_length,2.5..6.9  Set to be NULL
petal_length,2.5..6.9 also set to be NULL
Intersection set :  {51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 1

...

petal_length,1.0..5.2 also set to be NULL
petal_length,1.0..5.3 also set to be NULL
petal_length,1.0..5.5 also set to be NULL
petal_length,1.0..5.5 also set to be NULL
petal_length,1.0..5.7 also set to be NULL
petal_length,1.0..5.8 also set to be NULL
petal_length,1.0..5.8 also set to be NULL
petal_length,1.0..6.0 also set to be NULL
petal_length,1.0..6.0 also set to be NULL
petal_length,1.0..6.2 also set to be NULL
petal_length,1.0..6.3 also set to be NULL
petal_length,1.0..6.5 also set to be NULL
petal_length,1.0..6.7 also set to be NULL
petal_length,1.0..6.8 also set to be NULL
Intersection set :  {134, 52, 54, 56, 57, 62, 64, 67, 68, 70, 72, 74, 79, 80, 81, 82, 83, 88, 89, 90, 91, 92, 93, 95, 96, 97, 98, 100}
NOT
sepal_length,4.3..6.3  Set to be NULL
sepal_length,4.3..7.9 also set to be NULL
sepal_length,4.3..6.3 also set to be NULL
sepal_length,4.3..6.5 also set to be NULL
sepal_length,4.3..6.5 also set to be NULL
sepal_length,4.3..6.7 also set to be NULL
sepal_length,4.3..6.8 als

sepal_length,4.3..6.2 also set to be NULL
sepal_length,4.3..6.2 also set to be NULL
sepal_length,4.3..6.3 also set to be NULL
sepal_length,4.3..6.5 also set to be NULL
sepal_length,4.3..6.5 also set to be NULL
sepal_length,4.3..6.7 also set to be NULL
sepal_length,4.3..6.8 also set to be NULL
sepal_length,4.3..6.8 also set to be NULL
sepal_length,4.3..7.0 also set to be NULL
sepal_length,4.3..7.0 also set to be NULL
sepal_length,4.3..7.2 also set to be NULL
sepal_length,4.3..7.2 also set to be NULL
sepal_length,4.3..7.3 also set to be NULL
sepal_length,4.3..7.5 also set to be NULL
sepal_length,4.3..7.7 also set to be NULL
sepal_length,4.3..7.8 also set to be NULL
petal_length,4.3..6.9 also set to be NULL
Intersection set :  {96}
NOT
sepal_width,2.2..2.7  Set to be NULL
sepal_width,2.2..4.4 also set to be NULL
sepal_width,2.2..2.7 also set to be NULL
sepal_width,2.2..2.8 also set to be NULL
sepal_width,2.2..2.8 also set to be NULL
sepal_width,2.2..3.0 also set to be NULL
sepal_width,2.2

petal_length,1.8..6.9 also set to be NULL
Intersection set :  {128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 51, 52, 53, 55, 57, 64, 67, 69, 71, 73, 77, 78, 79, 84, 85, 86, 87, 92, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127}
NOT
petal_width,0.6..2.5  Set to be NULL
petal_width,0.6..2.5 also set to be NULL
Intersection set :  {128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 51, 52, 53, 55, 57, 64, 67, 69, 71, 73, 77, 78, 79, 84, 85, 86, 87, 92, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127}
NOT
petal_width,0.5..2.5  Set to be NULL
petal_width,0.5..2.5 also set to be NULL
Intersection set :  {128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 14

sepal_width,2.2..3.3 also set to be NULL
sepal_width,2.2..3.5 also set to be NULL
sepal_width,2.2..3.5 also set to be NULL
sepal_width,2.2..3.7 also set to be NULL
sepal_width,2.2..3.8 also set to be NULL
sepal_width,2.2..3.8 also set to be NULL
sepal_width,2.2..4.0 also set to be NULL
sepal_width,2.2..4.0 also set to be NULL
sepal_width,2.2..4.2 also set to be NULL
sepal_width,2.2..4.3 also set to be NULL
Intersection set :  {128, 129, 130, 131, 133, 136, 138, 139, 140, 141, 142, 143, 144, 146, 147, 148, 150, 71, 78, 84, 102, 103, 104, 105, 108, 109, 111, 112, 113, 114, 116, 117, 121, 124, 125, 126, 127}
NOT
petal_length,1.0..6.5  Set to be NULL
petal_length,1.0..6.5 also set to be NULL
petal_length,1.0..6.7 also set to be NULL
petal_length,1.0..6.8 also set to be NULL
Intersection set :  {128, 129, 130, 131, 133, 136, 138, 139, 140, 141, 142, 144, 146, 147, 148, 78, 84, 103, 104, 105, 108, 109, 111, 112, 113, 114, 116, 117, 121, 124, 125, 126, 127}
NOT
sepal_length,6.0..7.9  Set to b

sepal_length,4.3..6.5 also set to be NULL
sepal_length,4.3..6.5 also set to be NULL
sepal_length,4.3..6.7 also set to be NULL
sepal_length,4.3..6.8 also set to be NULL
sepal_length,4.3..6.8 also set to be NULL
sepal_length,4.3..7.0 also set to be NULL
sepal_length,4.3..7.0 also set to be NULL
sepal_length,4.3..7.2 also set to be NULL
sepal_length,4.3..7.2 also set to be NULL
sepal_length,4.3..7.3 also set to be NULL
sepal_length,4.3..7.5 also set to be NULL
sepal_length,4.3..7.7 also set to be NULL
sepal_length,4.3..7.8 also set to be NULL
petal_length,4.3..6.9 also set to be NULL
Intersection set :  {112, 117}
NOT
petal_length,5.3..6.9  Set to be NULL
sepal_length,5.3..7.9 also set to be NULL
petal_length,5.3..6.9 also set to be NULL
Intersection set :  {112, 117}
NOT
sepal_width,3.0..4.4  Set to be NULL
sepal_width,3.0..4.4 also set to be NULL
sepal_width,3.0..4.4 also set to be NULL
Intersection set :  {112, 117}
NOT
petal_length,1.0..5.5  Set to be NULL
petal_length,1.0..5.5 also s

In [68]:
print("Time to run the algorithm (for Upper Approximation): ", round(elapsed_time, 3), "Sec")
print("\n")
print(*final_rules, sep='\n')

with open('../data/Iris/Rules/Iris_15_attcon_upper.txt', 'w') as f:
    for item in final_rules:
        f.write("%s\n" % item)

Time to run the algorithm (for Upper Approximation):  9.562 Sec


['(sepal_width,3.5..3.3) & (sepal_length,4.3..4.8) -> (class,Iris-setosa)']
['(sepal_length,6.0..7.9) & (petal_length,1.0..4.2) & (sepal_width,2.2..2.7) & (petal_length,4.5..6.9) -> (class,Iris-versicolor)']
['(petal_width,2.0..1.9) & (sepal_length,4.3..6.3) -> (class,Iris-viginica)']
